In [2]:
import yfinance as yf

token = 'BTC-USD'

data = yf.download(token, interval='1d', period='max')
data = data.droplevel('Ticker', axis=1)
data.to_csv('BTC-USD.csv')
data

C:\Users\ivoto\AppData\Local\Temp\ipykernel_7124\3976575668.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(token, interval='1d', period='max')
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Date,,,,,
2014-09-17,457.334015,468.174011,452.421997,465.864014,21056800
2014-09-18,424.440002,456.859985,413.104004,456.859985,34483200
2014-09-19,394.795990,427.834991,384.532013,424.102997,37919700
2014-09-20,408.903992,423.295990,389.882996,394.673004,36863600
2014-09-21,398.821014,412.425995,393.181000,408.084991,26580100
...,...,...,...,...,...
2025-08-03,114217.671875,114747.421875,111943.804688,112525.804688,48099615826
2025-08-04,115071.882812,115729.468750,114130.406250,114223.921875,35783028986
2025-08-05,114141.445312,115117.437500,112701.109375,115072.187500,61039182286


In [4]:
import pandas as pd
from ta.trend import EMAIndicator, MACD
from ta.momentum import RSIIndicator

# Carga tus datos en 'data' con columna 'Close'
data = pd.read_csv('BTC-USD.csv')

# EMAs
data['ema20'] = EMAIndicator(data['Close'], window=20).ema_indicator()
data['ema50'] = EMAIndicator(data['Close'], window=50).ema_indicator()

# RSIs
data['rsi10']  = RSIIndicator(data['Close'], window=10).rsi()
data['rsi30']  = RSIIndicator(data['Close'], window=30).rsi()
data['rsi200'] = RSIIndicator(data['Close'], window=200).rsi()

# MACD
macd_ind = MACD(data['Close'], window_fast=12, window_slow=26, window_sign=9)
data['macd']   = macd_ind.macd()
data['signal'] = macd_ind.macd_signal()
data['hist']   = macd_ind.macd_diff()

# Señal de subida de precio
data['Price-Up'] = (data['Close'].shift(-1) > data['Close']).astype(int)

# Limpieza y exportación
data.dropna(inplace=True)
data.to_csv('BTC-I.csv', index=False)
data
print(data.tail())


            Date          Close           High            Low           Open  \
3973  2025-08-03  114217.671875  114747.421875  111943.804688  112525.804688   
3974  2025-08-04  115071.882812  115729.468750  114130.406250  114223.921875   
3975  2025-08-05  114141.445312  115117.437500  112701.109375  115072.187500   
3976  2025-08-07  117496.898438  117676.906250  114279.710938  115030.054688   
3977  2025-08-09  116776.765625  116975.523438  116379.140625  116688.750000   

           Volume          ema20          ema50      rsi10      rsi30  \
3973  48099615826  115942.420815  112925.492746  41.625144  53.243667   
3974  35783028986  115859.512434  113009.664905  45.982605  54.347019   
3975  61039182286  115695.886994  113054.048451  42.172664  52.939359   
3976  64051649681  115867.411893  113228.277862  56.586286  57.086102   
3977  54840418304  115954.017011  113367.434245  53.411643  55.990758   

         rsi200        macd       signal        hist  Price-Up  
3973  54.515007

# Stream_Crypto_Data 

In [7]:
import json 
import websocket
import pandas as pd
import numpy as np

coin = 'btcusdt@kline_1m'
socket = 'wss://stream.binance.com:9443/stream?streams=' + coin
print(socket)


ModuleNotFoundError: No module named 'websocket'

In [ ]:
def extract_ohlcv(source):
    event_time = pd.to_datetime(source['data']['E'], unit='ms')
    open = source['data']['k']['o']
    high = source['data']['k']['h']
    low = source['data']['k']['l']
    close = source['data']['k']['c']   
    volume = source['data']['k']['v']

    data = {'Open': open, 'High': high, 'Low': low, 'Close': close, 'Volume': volume}

    df = pd.DataFrame(data, index=[event_time])
    df.index.name = 'timestamp'
    df = df.astype(float)
    df = df.reset_index()
    print(df)
    return df

def on_message(ws, message):
    message = json.loads(message)
    extract_ohlcv(message)

In [ ]:
ws = websocket.WebSocketApp(socket, on_message=on_message)
ws.run_forever()

                timestamp      Open     High       Low     Close    Volume
0 2025-02-27 17:53:18.014  84573.53  84593.7  84494.26  84494.26  10.41565
                timestamp      Open     High       Low     Close    Volume
0 2025-02-27 17:53:20.015  84573.53  84593.7  84494.26  84494.27  10.43499
                timestamp      Open     High       Low     Close    Volume
0 2025-02-27 17:53:22.029  84573.53  84593.7  84494.26  84494.27  10.61498


True